In [1]:
# Import pandas library and get main file
import pandas as pd

# get sales data
Sales = pd.read_csv("1A_SampleSales.csv")  

In [2]:
Sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44167 entries, 0 to 44166
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ItemID                  44167 non-null  object 
 1   year                    44167 non-null  int64  
 2   WeekIdentifier          44167 non-null  int64  
 3   SalesChannel            44167 non-null  object 
 4   Territory               44167 non-null  object 
 5   NewBasePrice            44167 non-null  float64
 6   TotalInventory          44167 non-null  int64  
 7   StoreInventory          44167 non-null  int64  
 8   WarehouseInventory      44167 non-null  int64  
 9   StockedStorePercentage  44167 non-null  float64
 10  SellingStoresRatio      44167 non-null  float64
 11  InboundInventory        44167 non-null  int64  
 12  SalesQuantity           44167 non-null  int64  
 13  ProjectedInventory      44167 non-null  int64  
dtypes: float64(3), int64(8), object(3)
mem

In [3]:
# Ensure NewBasePrice is clean numeric with 2 decimals
Sales["NewBasePrice"] = (
    Sales["NewBasePrice"]
    .astype(str)
    .str.replace(r"[$,]", "", regex=True)  # remove $ and commas
    .astype(float)
    .round(2)
)

# Ensure SellingStoresRaio is numeric (remove commas or spaces just in case)
Sales["SellingStoresRatio"] = (
    Sales["SellingStoresRatio"]
    .astype(str)
    .str.replace(r"[,\s]", "", regex=True)  # remove commas and spaces
    .astype(float)
)

# Ensure SalesQuantity is numeric (remove commas or spaces just in case)
Sales["SalesQuantity"] = (
    Sales["SalesQuantity"]
    .astype(str)
    .str.replace(r"[,\s]", "", regex=True)  # remove commas and spaces
    .astype(float)
)

# Create revenues column
Sales["revenues"] = (Sales["NewBasePrice"] * Sales["SalesQuantity"]).round(2)


In [4]:
# Get Summary stats of key variables to check for obvious erros - like negative numbers
Sales[["SalesQuantity", "NewBasePrice", "SellingStoresRatio"]].describe()

,SalesQuantity,NewBasePrice,SellingStoresRatio
count,44167.000000,44167.000000,44167.000000
mean,12.583264,19.448836,0.095677
std,31.723881,3.840044,0.124611
min,0.000000,10.880000,0.000000
25%,3.000000,16.725000,0.036400
50%,5.000000,19.190000,0.066900
75%,8.000000,21.920000,0.095900
max,957.000000,46.240000,1.083200


**No negative Numbers.**

**The max numbers for SalesQuantiy and NewbasePrice look suspicious.  Will take a closer look during EDA Stage.**

In [5]:
# get items data
Items = pd.read_csv("1B_Items.csv")  

Items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ItemID       9205 non-null   object
 1   Period       9205 non-null   object
 2   Category     9205 non-null   object
 3   SubCategory  9205 non-null   object
 4   Type         9205 non-null   object
 5   SubType      9205 non-null   object
dtypes: object(6)
memory usage: 431.6+ KB


In [6]:
# Create a new dataframe with Category and Subcategory added
# Need to be able to provide new CEO with top line overview categories and subcategories are performing

Sales_with_items = Sales.merge(
    Items[["ItemID", "Category", "SubCategory"]],
    on="ItemID",
    how="left"
)


In [7]:
# Drop inventory-related columns, they are not relevant to the proect
Sales_with_items = Sales_with_items.drop(
    columns=[
        "TotalInventory",
        "StoreInventory",
        "WarehouseInventory",
        "StockedStorePercentage",
        "InboundInventory",
        "ProjectedInventory"
    ]
)

#Check columns are dropped
Sales_with_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44167 entries, 0 to 44166
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ItemID              44167 non-null  object 
 1   year                44167 non-null  int64  
 2   WeekIdentifier      44167 non-null  int64  
 3   SalesChannel        44167 non-null  object 
 4   Territory           44167 non-null  object 
 5   NewBasePrice        44167 non-null  float64
 6   SellingStoresRatio  44167 non-null  float64
 7   SalesQuantity       44167 non-null  float64
 8   revenues            44167 non-null  float64
 9   Category            44167 non-null  object 
 10  SubCategory         44167 non-null  object 
dtypes: float64(4), int64(2), object(5)
memory usage: 3.7+ MB


In [8]:
# Count missing values per column
Sales_with_items.isnull().sum()

ItemID                0
year                  0
WeekIdentifier        0
SalesChannel          0
Territory             0
NewBasePrice          0
SellingStoresRatio    0
SalesQuantity         0
revenues              0
Category              0
SubCategory           0
dtype: int64

**Great, there are no missing values**

In [9]:
# Count total duplicate rows
Sales_with_items.duplicated().sum()

0

**Good, no duplicates either.**

In [10]:
# Exclude E-Commerce sales data from data set
Sales_with_items = Sales_with_items.loc[Sales_with_items["SalesChannel"] != "E-commerce"]

In [11]:
# Determine the top selling items that contribute to 70-80% of revenue ($s).  

# Focus on the last year sales, to reflect more recent trends
sales_2019 = Sales_with_items[Sales_with_items["year"] == 2019]

#Calculate Total 2019 revenue for each item
item_sales = (
    sales_2019.groupby(["ItemID", "Category", "SubCategory"], as_index=False)["revenues"]
    .sum()
    .rename(columns={"revenues": "TotalRevenue"})
)

#Sort items by Total 2019 Revenue, Highest to Lowest
item_sales = item_sales.sort_values(by="TotalRevenue", ascending=False).reset_index(drop=True)

#Calculate cummulative contribution
total_revenue = item_sales["TotalRevenue"].sum()
item_sales["CumulativeShare"] = item_sales["TotalRevenue"].cumsum() / total_revenue

#Filter to the top items - check both for 70% and 80%
top_70 = item_sales[item_sales["CumulativeShare"] <= 0.70] 
top_80 = item_sales[item_sales["CumulativeShare"] <= 0.80] 

In [12]:
# quick sanity checks
sum70 = top_70["TotalRevenue"].sum()
sum80 = top_80["TotalRevenue"].sum()

print(f"Top-70 count = {len(top_70)}, total revenue = ${sum70:,.2f}, coverage = {sum70/total_revenue:.1%}")
print(f"Top-80 count = {len(top_80)}, total revenue = ${sum80:,.2f}, coverage = {sum80/total_revenue:.1%}")

Top-70 count = 143, total revenue = $1,785,234.95, coverage = 69.9%
Top-80 count = 202, total revenue = $2,041,091.15, coverage = 79.9%


In [13]:
# additional sanity check
print(f"Top-70 items account for {len(top_70)/len(item_sales):1%} of all items")
print(f"Top-80 items account for {len(top_80)/len(item_sales):1%} of all items")

Top-70 items account for 20.000000% of all items
Top-80 items account for 28.251748% of all items


**I will will choose the larger group, items contributing to 80% of all revenue**

In [14]:
# Filter data to focus only on items that represent Top 80% of Revenues

# IDs of top-80 items
top80_ids = top_80["ItemID"].unique()

# Filter Sales_with_items for only those items
Sales_top80 = Sales_with_items[Sales_with_items["ItemID"].isin(top80_ids)].copy()

# Quick sanity checks
print(f"Rows kept (for top-80 items): {len(Sales_top80):,}")
print(f"Unique items kept: {len(Sales_top80['ItemID'].unique()):,}")

# Save top-80 items subset
Sales_top80.to_csv("1C_Sales_top80_items_all_dates.csv", index=False)


Rows kept (for top-80 items): 4,798
Unique items kept: 202


In [15]:
#do a quick check of the data, check for variability in NewBasePrice and SellingStoresRatio
#also check counts of observances by Unique ItemID
item_summary = Sales_top80.groupby("ItemID").describe()
item_summary = (
    Sales_top80.groupby("ItemID")[["SalesQuantity", "NewBasePrice", "SellingStoresRatio"]]
    .agg(["count", "mean", "min", "max"])
)

item_summary.sort_values(by=("SalesQuantity", "count"), ascending=False, inplace=True)

item_summary.head(100)

SalesQuantity                        NewBasePrice                    \
                  count       mean  min    max        count       mean    min   
ItemID                                                                          
yYAY5pI85            48  16.895833  2.0   56.0           48  22.413333  18.08   
Sp6FmlM84            47  69.659574  2.0  165.0           47  15.570638  12.01   
hQjJxUI52            46  42.543478  0.0  130.0           46  12.800000  12.80   
NWpKkFc27            44  98.840909  6.0  487.0           44  19.783636  15.38   
52EVihM31            44  45.772727  2.0  416.0           44  22.860000  22.86   
...                 ...        ...  ...    ...          ...        ...    ...   
a9hB0UQ19            22  22.409091  0.0  107.0           22  19.960000  19.96   
YhDFyMk67            21  61.047619  7.0  305.0           21  24.230000  24.23   
b67FgJw34            21  30.619048  1.0   92.0           21  15.518095  12.23   
Ehc_Blo65            21  23.428571  4.0   65.0           21  15.298095  11.56   
T1eq7WQ60            21  62.190476  6.0  275.0           21  24.620000  24.62   

                 SellingStoresRatio                            
             max              count      mean     min     max  
ItemID                                                         
yYAY5pI85  27.33                 48  0.382625  0.0569  1.0335  
Sp6FmlM84  19.01                 47  0.343519  0.0409  0.7002  
hQjJxUI52  12.80                 46  0.305789  0.0000  0.7478  
NWpKkFc27  23.96                 44  0.311327  0.0361  1.0152  
52EVihM31  22.86                 44  0.142227  0.0070  0.7096  
...          ...                ...       ...     ...     ...  
a9hB0UQ19  19.96                 22  0.102532  0.0168  0.3566  
YhDFyMk67  24.23                 21  0.279386  0.0387  1.0272  
b67FgJw34  18.78                 21  0.143281  0.0000  0.3245  
Ehc_Blo65  19.01                 21  0.437600  0.0362  1.0279  
T1eq7WQ60  24.62                 21  0.275600  0.0442  0.7396  

[100 rows x 12 columns]

**On a quick scan, there is enough variability in the data to allow prediction**